# Prerequisites

- Host OS: Ubuntu 20.04 lts
- Using Docker Image 'mltooling/ml-workspace-gpu' (docker pull mltooling/ml-workspace-gpu)
- Single Nvidia GPU (RTX 3080)

# 0. Import libraries

In [2]:
import bert_ensemble_functions
from datasets import load_dataset

import transformers
import datasets
import huggingface_hub
import pyarrow
import torch
print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)
print(torch.__version__)

4.22.1
2.4.0
0.9.1
9.0.0
1.9.0+cu111


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    device = torch.device("cpu")
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [5]:
#### The number of CPU cores
!grep -c processor /proc/cpuinfo

20


In [6]:
#### GPU information
!nvidia-smi

Tue Nov 22 00:25:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 58%   42C    P5    52W / 370W |    253MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Customize Train Strategy

In [7]:
num_cpus = 16
num_gpus = 1
seed = 1234
model_name = "xlm-roberta-base" # bert-base-multilingual-cased, klue/roberta-base, bert-base-cased, etc.

text_column = 'past_history'
label_column = 'label'
id_column = 'id'
custom_dir = f"sev_{text_column}_ensemble"

train_proportion = 0.5 # train set : eval set = 5 : 5
do_hpo = True

# If you want to search best hyperparameters using ray tune, parameters below should be set
n_trials = 5
std = 0.1
patience = 5

# 2. Import Data

2 files are needed (`{data_name}_train.csv` and `{data_name}_test.csv`) in your data directory (in this case, `data_splited/`).

In [8]:
data_name = "cardiovascular_sev_dataset" 

dataset = load_dataset('csv', data_files={'train': f'../data_split/{data_name}_train.csv',
                                          'test': f'../data_split/{data_name}_test.csv'})
dataset

Using custom data configuration default-9aa18915b5f32f1a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 3756
    })
    test: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 940
    })
})

# 3. Data Preprocessing

In [9]:
train_dataset, eval_dataset, test_dataset = bert_ensemble_functions.preprocessing(dataset = dataset,
                                                                                  text_column = text_column, 
                                                                                  label_column = label_column,
                                                                                  id_column = id_column,
                                                                                  model_name = model_name,
                                                                                  train_proportion = train_proportion,
                                                                                  seed = seed,
                                                                                  custom_tokenizer_dir = custom_dir)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-1c0e7ead3223d9bb.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-53953f976f8d532a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-e7f16e34a4d4f3cb.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-4104c11161fb780b.arrow


Removing rows with missing value...
Done. (1/4)
Removing special characters...
Done. (2/4)
Tokenining the text column...


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-aa34132bc605a373.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-722859b2adb147eb.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0ff6e6cdc3e42c22.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0ff6e6cdc3e42c22.arrow


Done. (3/4)
Spliting train-evaluation-test set...
Done. (4/4)


# 4. Modeling

In [10]:
trainer = bert_ensemble_functions.modeling(train_dataset=train_dataset,
                                           eval_dataset=eval_dataset,
                                           model_name='xlm-roberta-base',
                                           num_gpus=num_gpus,
                                           num_cpus=num_cpus,
                                           seed=seed,
                                           output_dir='./output',
                                           logging_dir="./logs",
                                           do_hpo=do_hpo,
                                           std = std,
                                           n_trials = n_trials,
                                           patience = patience,
                                           hpo_result_dir = "./hpo-results",
                                           hpo_result_dir_subfolder_name = 'tune_transformer_pbt'
                                          )



loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

== Status ==
Current time: 2022-11-22 00:25:42 (running for 00:00:00.17)
Memory usage on this node: 9.9/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.26 GiB heap, 0.0/7.13 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_32bd6_00000 | RUNNING  | 172.17.0.3:3928549 |  0.186633 | 2.75091e-05 |              8 |           17 |
| _objective_32bd6_00001 | PENDING  |                    |  0.287442 | 4.50373e-05 |              8 |  

== Status ==
Current time: 2022-11-22 00:26:14 (running for 00:00:32.29)
Memory usage on this node: 14.5/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.26 GiB heap, 0.0/7.13 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_32bd6_00000 | RUNNING  | 172.17.0.3:3928549 |  0.186633 | 2.75091e-05 |              8 |           17 |
| _objective_32bd6_00001 | PENDING  |                    |  0.287442 | 4.50373e-05 |              8 | 

== Status ==
Current time: 2022-11-22 00:26:45 (running for 00:01:02.94)
Memory usage on this node: 14.9/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.26 GiB heap, 0.0/7.13 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (1 PAUSED, 3 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+------

== Status ==
Current time: 2022-11-22 00:27:05 (running for 00:01:22.96)
Memory usage on this node: 14.9/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.26 GiB heap, 0.0/7.13 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (1 PAUSED, 3 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+------

...

Result for _objective_32bd6_00001:
  date: 2022-11-22_00-53-53
  done: false
  episodes_total: 0
  epoch: 1.71
  eval_accuracy: 0.8248136315228967
  eval_f1: 0.6808923375363725
  eval_loss: 0.5182244181632996
  eval_objective: 1.5057059690592691
  eval_runtime: 12.5538
  eval_samples_per_second: 149.597
  eval_steps_per_second: 18.719
  experiment_id: 1d008982c96a43038d5d2e6ce20ec600
  hostname: 3481a8a2ae33
  iterations_since_restore: 2
  node_ip: 172.17.0.3
  objective: 3.0114119381185382
  pid: 3928549
  time_since_restore: 94.13283061981201
  time_this_iter_s: 46.54751777648926
  time_total_s: 373.8215525150299
  timestamp: 1669078433
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 2
  trial_id: 32bd6_00001
  warmup_time: 0.0017666816711425781
  
== Status ==
Current time: 2022-11-22 00:53:58 (running for 00:28:16.73)
Memory usage on this node: 15.8/31.1 GiB
PopulationBasedTraining: 9 checkpoints, 2 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0

== Status ==
Current time: 2022-11-22 00:54:18 (running for 00:28:36.74)
Memory usage on this node: 15.8/31.1 GiB
PopulationBasedTraining: 9 checkpoints, 2 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.26 GiB heap, 0.0/7.13 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PAUSED, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------

== Status ==
Current time: 2022-11-22 00:54:39 (running for 00:28:56.75)
Memory usage on this node: 15.8/31.1 GiB
PopulationBasedTraining: 9 checkpoints, 2 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.26 GiB heap, 0.0/7.13 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PAUSED, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------

2022-11-22 00:54:39,766	INFO tune.py:758 -- Total run time: 1737.63 seconds (1737.30 seconds for the tuning loop).


AttributeError: 'NoneType' object has no attribute 'save_pretrained'

In [11]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

Step,Training Loss,Validation Loss,Accuracy,F1,Objective
50,No log,0.852671,0.283813,0.440748,0.724561
100,No log,0.465065,0.750266,0.682464,1.432731
150,No log,0.385477,0.882854,0.809028,1.691882
200,No log,0.317166,0.885517,0.814495,1.700012
250,No log,0.476246,0.894036,0.817599,1.711635
300,No log,0.399619,0.892439,0.821869,1.714308
350,No log,0.458469,0.909478,0.838710,1.748188
400,No log,0.472082,0.905751,0.835349,1.741100
450,No log,0.646326,0.907881,0.827173,1.735054
500,0.305900,0.899089,0.891906,0.783813,1.675719


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1878
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/transformers/trainer.py:1808: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignor

TrainOutput(global_step=696, training_loss=0.23214142075900374, metrics={'train_runtime': 632.234, 'train_samples_per_second': 35.645, 'train_steps_per_second': 1.101, 'total_flos': 5923682300375040.0, 'train_loss': 0.23214142075900374, 'epoch': 11.99})

In [12]:
# save the pretrained model
trainer.model.save_pretrained(custom_dir)

Configuration saved in sev_past_history_ensemble/config.json
Model weights saved in sev_past_history_ensemble/pytorch_model.bin


In [13]:
df = bert_ensemble_functions.evaluation(trainer = trainer,
                                        eval_dataset = eval_dataset,
                                        text_column_name = text_column
                                        )
df

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1878
  Batch size = 8


,id,text,past_history_pred_0,past_history_pred_1,label,pred
0,17370,상기 53세 남환 HTN DM dyslipidemia 과거력 있는 분으로 이삿짐센터...,4.199219,-4.375000,0,0
1,18264,상기 55세남환 특이적 내과 과거력 없는자로 금일 테니스를 30분정도 친후 내원 한...,-3.250000,3.673828,1,1
2,15281,상기 여환 뇌졸중 당뇨 ESRD 과거력 있고 CVA로 서울대 병원 신경과 fu ...,3.642578,-3.609375,0,0
3,51130,Chief complaint aphasia Ltside weakness Clear...,4.152344,-4.226562,1,0
4,46160,상환 DMold CVACAOD 3vd sp stent insertion 과거력 있는...,4.164062,-4.242188,0,0
...,...,...,...,...,...,...
1873,38025,PHxHTNDMpulTBchepatitis alcohol smoking ro CA...,4.187500,-4.257812,0,0
1874,9988,상기 여환은 1993년 12월 ESRD 진단 후 최규헌 선생님 fu 하면서 CAPD...,4.050781,-4.066406,0,0
1875,40275,상기 여환 HTN hypercholestorolemia 병력 있는 분으로 내원 10...,4.117188,-4.289062,0,0
1876,49338,상환 특이 내과적 과거력 없던 분으로 내원 6개월전부터 간헐적 흉통 있었다고 하며 ...,4.179688,-4.375000,0,0


In [16]:
# Check the classification result of each XLM-RoBERTa Model 
from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)

print("Confusion Matrix")
print(confusion_matrix(df.label, df.pred))
print("-------------------------")
accuracy = accuracy_score(df.label, df.pred)
f1 = f1_score(df.label, df.pred)
recall = recall_score(df.label, df.pred)
precision = precision_score(df.label, df.pred)
print(f"Accuracy: {accuracy}")
print(f"F1 score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")

Confusion Matrix
[[1307   41]
 [ 162  368]]
-------------------------
Accuracy: 0.8919062832800851
F1 score: 0.7838125665601703
Recall: 0.6943396226415094
Precision: 0.8997555012224939


In [17]:
df.to_csv(f'./{text_column}_bert_result_df.csv', index=False)